In [46]:
import pickle
import numpy as np
import pandas as pd

In [47]:
X,y_hr,y_participant,y_activity,y_time = pickle.load(open('../data/dalia_individual_windows.p','rb'))

In [48]:
def get_train_test_val_data(X_hr,y_participant,y_time,y_activity):
    all_data = pd.DataFrame({'x':np.arange(X_hr.shape[0]),'y':y_participant,'time':y_time,'activity':y_activity})
    train_percentage = .6
    val_percentage = train_percentage+.1
    def split_data(df):
        df = df.sort_values('time').reset_index(drop=True)
        n = df.shape[0]
        train_index = df['x'].values[:int(n*train_percentage)]
        val_index = df['x'].values[int(n*train_percentage):int(n*val_percentage)]
        test_index = df['x'].values[int(n*val_percentage):]
        return pd.DataFrame({'train':[list(train_index)],'val':[list(val_index)],'test':[list(test_index)]})

    index_df = all_data.groupby(['y','activity'],as_index=False).apply(split_data)

    from functools import reduce
    train_index = np.array(reduce(lambda a,b:a+b,index_df['train'].values))
    val_index = np.array(reduce(lambda a,b:a+b,index_df['val'].values))
    test_index = np.array(reduce(lambda a,b:a+b,index_df['test'].values))


    train_x,train_y = X_hr[train_index],y_participant[train_index]
    val_x,val_y = X_hr[val_index],y_participant[val_index]
    test_x,test_y = X_hr[test_index],y_participant[test_index]
    return train_x,train_y,val_x,val_y,test_x,test_y 

In [49]:
n_timesteps = 256
n_channels = 4
train_x,train_y,val_x,val_y,test_x,test_y = get_train_test_val_data(X[:,:,:n_channels],y_participant,y_time,y_activity)

In [50]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib inline
from keras.layers import Conv1D,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input,GRU
from keras.models import Model, Sequential
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats.mstats import winsorize

4 Physical GPUs, 1 Logical GPU


In [51]:
def get_model(input_shape=(256,4),act='relu',loss="categorical_crossentropy",opt='adam',
              n_classes=350,n_output = 15):
    
    
    model =  Sequential()
    model.add(Input(shape=input_shape))
    model.add(Reshape(input_shape))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='selu',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(BatchNormalization())
#     model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(BatchNormalization())
#     model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(Bidirectional(GRU(10,activation='relu',return_sequences=True)))
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_output,activation=None))
    
    input_ = Input(shape=input_shape)
    embedding = model(input_)
    final_embedding = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1),name='feature')(embedding)
    
#     class_embedding = Dense(n_output,activation='sigmoid')(final_embedding)
    y_output = Dense(n_output,activation='softmax',name='output')(final_embedding) 
    model1 = Model(input_,[y_output,final_embedding])
    model1.compile(loss={'output':tf.keras.losses.SparseCategoricalCrossentropy(),
                        'feature':tfa.losses.TripletSemiHardLoss()},optimizer=opt,
                   loss_weights={'output':1,'feature':1})
    return model1

model1 = get_model(input_shape=(n_timesteps,1))
model1.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        [(None, 256, 1)]          0         
_________________________________________________________________
sequential_14 (Sequential)   (None, 15)                336665    
_________________________________________________________________
feature (Lambda)             (None, 15)                0         
_________________________________________________________________
output (Dense)               (None, 15)                240       
Total params: 336,905
Trainable params: 336,305
Non-trainable params: 600
_________________________________________________________________


In [52]:
n_classes = 300
n_output = len(np.unique(y_participant))
model = get_model(input_shape=(n_timesteps,n_channels),n_classes=n_classes,n_output=n_output) 
model.summary()
from keras.models import load_model
filepath = '../model_files/base_cnn_60_seconds_raw_ppg_acl_classification.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,[train_y,train_y],validation_data=(val_x,[val_y,val_y]), epochs=400, 
                    batch_size=600,callbacks=callbacks_list,shuffle=True,verbose=1)

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 256, 4)]          0         
_________________________________________________________________
sequential_15 (Sequential)   (None, 15)                339665    
_________________________________________________________________
feature (Lambda)             (None, 15)                0         
_________________________________________________________________
output (Dense)               (None, 15)                240       
Total params: 339,905
Trainable params: 339,305
Non-trainable params: 600
_________________________________________________________________
Epoch 1/400
47/48 [============================>.] - ETA: 0s - loss: 3.4593 - output_loss: 2.4633 - feature_loss: 0.9960
Epoch 00001: val_loss improved from inf to 3.69266, saving model to ../model_files/base_cnn_60_seconds_raw_ppg_acl_classificati

In [43]:
model.load_weights(filepath)

In [44]:
y_pred = model.predict(test_x)[0].argmax(axis=1)

In [45]:
from sklearn.metrics import classification_report

print(classification_report(test_y,y_pred),accuracy_score(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.39      0.33      0.35      1055
           1       0.44      0.26      0.33       999
           2       0.54      0.30      0.38       886
           3       0.36      0.44      0.40      1013
           4       0.66      0.49      0.56      1075
           5       0.58      0.29      0.39       892
           6       0.35      0.46      0.40       445
           7       0.48      0.37      0.42      1016
           8       0.54      0.30      0.38      1070
           9       0.50      0.63      0.56      1013
          10       0.36      0.56      0.43       987
          11       0.47      0.59      0.52       902
          12       0.51      0.60      0.55       854
          13       0.45      0.34      0.38       964
          14       0.30      0.58      0.39      1045

    accuracy                           0.44     14216
   macro avg       0.46      0.44      0.43     14216
weighted avg       0.46   